In [2]:
%pip install pytrends pandas numpy pyarrow

  Using cached pytrends-4.9.2-py3-none-any.whl.metadata (13 kB)
Using cached pytrends-4.9.2-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 3.1 MB/s  0:00:09m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 6.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pytrends]1/3 [lxml]
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pytrends.request import TrendReq
import pandas as pd
import numpy as np
import time

/Users/okuran/Desktop/thesis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
pytrends = TrendReq(hl="en-US", tz=360)

# Global config
TIMEFRAME = "2016-01-01 2025-07-31"
ANCHOR = "weather"
SLEEP_SEC = 2.0  

KEYWORDS = [
    "syphilis", "gonorrhea", "chlamydia",
    "syphilis symptoms", "gonorrhea symptoms", "chlamydia symptoms",
    "STD test", "HIV testing", "STD clinic"
]
KEYWORDS = sorted(KEYWORDS)

In [5]:
dma_map = pd.read_csv("DMA_FIPS_County_Mapping.csv")
dma_map.head()

,FIPS,STATE,COUNTY,STATEFP,CNTYFP,CNTYTVHH,DMAINDEX,DMA,GOOGLE_DMA
0,1001,AL,Autauga,1,1,16610,116,MONTGOMERY (AL) - SELMA (AL) ...,Montgomery (Selma) AL
1,1003,AL,Baldwin,1,3,58950,62,MOBILE (AL) - PENSACOLA (FL) - FT. WALTON (FL)...,Mobile AL-Pensacola (Ft. Walton Beach) FL
2,1005,AL,Barbour,1,5,10570,126,COLUMBUS (GA) ...,Columbus GA
3,1007,AL,Bibb,1,7,7820,40,BIRMINGHAM (AL) - ANNISTON (AL) - TUSCALOOSA (...,Birmingham AL
4,1009,AL,Blount,1,9,20000,40,BIRMINGHAM (AL) - ANNISTON (AL) - TUSCALOOSA (...,Birmingham AL


In [6]:
all_dma_results = []

for kw in KEYWORDS:
    print(f"Fetching {kw} ...")
    pytrends.build_payload([kw], timeframe=TIMEFRAME, geo="US", gprop="")
    df = pytrends.interest_by_region(resolution="DMA", inc_low_vol=True, inc_geo_code=True)
    df = df.reset_index().rename(columns={"geoName":"dma_name", "geoCode":"dma_code", kw:"gt_value"})
    df["keyword"] = kw
    all_dma_results.append(df)
    time.sleep(SLEEP_SEC)

dma_gt = pd.concat(all_dma_results, ignore_index=True)
dma_gt.head()


Fetching HIV testing ...


TooManyRequestsError: The request failed: Google returned a response with code 429